# Задание по теме «Коллаборативная фильтрация»

Преподаватель: Наталья Баданина, Юлия Пономарева, Егор Шишковец

Пакет SURPRISE:

    * используйте данные MovieLens 1M,
    * можно использовать любые модели из пакета,
    * получите RMSE на тестовом сете 0,87 и ниже.

Комментарий преподавателя:

В домашнем задании на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе Cross-validation (5 фолдов), а не на отложенном датасете.


In [71]:
from surprise import KNNWithMeans, KNNBasic, KNNWithZScore, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [72]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [73]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [74]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on = 'movieId').reset_index(drop = True)
movies_with_ratings.dropna(inplace = True)

In [75]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [76]:
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [77]:
cnt_ratings_per_user=dataset.groupby('uid').rating.count().reset_index().rename(columns={'rating': 'cnt_ratings'})
cnt_ratings_per_user[cnt_ratings_per_user.cnt_ratings < 5].uid.count()

0

In [78]:
cnt_ratings_per_user.describe()

,uid,cnt_ratings
count,610.000000,610.000000
mean,305.500000,165.304918
std,176.236111,269.480584
min,1.000000,20.000000
25%,153.250000,35.000000
50%,305.500000,70.500000
75%,457.750000,168.000000
max,610.000000,2698.000000


In [79]:
cnt_ratings_per_movie=dataset.groupby('iid').rating.count().reset_index().rename(columns={'rating': 'cnt_ratings'})

In [80]:
cnt_ratings_per_movie.describe()

,cnt_ratings
count,9719.000000
mean,10.375141
std,22.406220
min,1.000000
25%,1.000000
50%,3.000000
75%,9.000000
max,329.000000


In [81]:
ratings[~ratings.movieId.isin(cnt_ratings_per_movie)].rating.min()

0.5

In [82]:
ratings[~ratings.movieId.isin(cnt_ratings_per_movie)].rating.max()

5.0

In [83]:
reader = Reader(rating_scale = (0.5, 5.0))
data = Dataset.load_from_df(dataset[~dataset.iid.isin(cnt_ratings_per_movie)], reader)

In [84]:
trainset, testset = train_test_split(data, test_size = 0.2, random_state = 13)

In [85]:
trainset.all_items()

range(0, 8957)

Алогоритм KNNBaseline

In [86]:
algo = KNNBaseline(k = 43, min_k = 5, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [87]:
test_pred = algo.test(testset)

In [88]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8634


0.8634021700796014